<a href="https://colab.research.google.com/github/hnfksdjjn/pipeline-automatizando-limpeza-de-dados/blob/main/pipeline_automatizado_limpesa_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color= red> Pipeline Automatizado Limpeza Dados







In [ ]:
import pandas as pd
import logging
from IPython.display import display

# Configuração do logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Função 1: Carregar dados
def load_data(file_path, delimiter=";"):
    """
    Carrega o conjunto de dados a partir de um arquivo CSV.

    Args:
        file_path (str): Caminho para o arquivo CSV.
        delimiter (str): Delimitador usado no arquivo CSV (padrão: ';').

    Returns:
        pd.DataFrame: DataFrame carregado.

    Raises:
        Exception: Se ocorrer um erro ao carregar os dados.
    """
    try:
        df = pd.read_csv(file_path, delimiter=delimiter)
        logging.info("Dados carregados com sucesso.")
        return df
    except Exception as e:
        logging.error(f"Erro ao carregar dados: {e}")
        raise

# Função 2: Tratar valores ausentes
def handle_missing_values(df, strategy="mean"):
    """
    Trata valores ausentes no DataFrame.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        strategy (str): Estratégia de preenchimento ('mean', 'median', 'mode', 'drop').

    Returns:
        pd.DataFrame: DataFrame com valores ausentes tratados.
    """
    try:
        if strategy == "mean":
            df.fillna(df.mean(numeric_only=True), inplace=True)
        elif strategy == "median":
            df.fillna(df.median(numeric_only=True), inplace=True)
        elif strategy == "mode":
            df.fillna(df.mode().iloc[0], inplace=True)
        elif strategy == "drop":
            df.dropna(inplace=True)
        else:
            raise ValueError("Estratégia inválida. Use 'mean', 'median', 'mode' ou 'drop'.")
        logging.info(f"Valores ausentes tratados com a estratégia '{strategy}'.")
        return df
    except Exception as e:
        logging.error(f"Erro ao tratar valores ausentes: {e}")
        raise

# Função 3: Remover duplicatas
def handle_duplicates(df, subset=None, keep="first"):
    """
    Remove ou ajusta duplicatas no DataFrame.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        subset (list): Colunas específicas para verificar duplicatas (opcional).
        keep (str): Quais duplicatas manter ('first', 'last' ou False).

    Returns:
        pd.DataFrame: DataFrame sem duplicatas.
    """
    try:
        before = len(df)
        df.drop_duplicates(subset=subset, keep=keep, inplace=True)
        after = len(df)
        logging.info(f"Removidos {before - after} registros duplicados.")
        return df
    except Exception as e:
        logging.error(f"Erro ao lidar com duplicatas: {e}")
        raise

# Função 4: Normalizar colunas numéricas
def normalize_columns(df):
    """
    Normaliza valores em todas as colunas numéricas para o intervalo [0, 1].

    Args:
        df (pd.DataFrame): DataFrame de entrada.

    Returns:
        pd.DataFrame: DataFrame com colunas normalizadas.
    """
    try:
        numeric_cols = df.select_dtypes(include=["number"]).columns
        for column in numeric_cols:
            df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
            logging.info(f"Coluna '{column}' normalizada.")
        return df
    except Exception as e:
        logging.error(f"Erro ao normalizar colunas: {e}")
        raise

# Função 5: Renomear colunas para melhorar a legibilidade
def rename_columns(df, columns_dict):
    """
    Renomeia as colunas para nomes mais legíveis.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        columns_dict (dict): Dicionário com os novos nomes das colunas.

    Returns:
        pd.DataFrame: DataFrame com as colunas renomeadas.
    """
    try:
        # Verifica se todas as colunas no dicionário existem no DataFrame
        missing_cols = [col for col in columns_dict if col not in df.columns]
        if missing_cols:
            logging.warning(f"As seguintes colunas não foram encontradas e não podem ser renomeadas: {missing_cols}")

        # Renomeia as colunas que existem no DataFrame
        df.rename(columns={col: columns_dict[col] for col in columns_dict if col in df.columns}, inplace=True)
        logging.info("Colunas renomeadas com sucesso.")
        return df
    except Exception as e:
        logging.error(f"Erro ao renomear as colunas: {e}")
        raise


# Função 6: Formatar os dados para facilitar a leitura
def format_data(df):
    """
    Melhora a formatação dos dados para torná-los mais legíveis e fáceis de interpretar.

    Args:
        df (pd.DataFrame): DataFrame de entrada.

    Returns:
        pd.DataFrame: DataFrame com formatação melhorada.
    """
    # Arredondar as colunas numéricas para 2 casas decimais
    df = df.round(2)

    # Converte as colunas 'Masculino' e 'Feminino' para formatos mais legíveis
    df['Sexo'] = df['Sexo'].replace({'M': 'Masculino', 'F': 'Feminino'})

    # Formatar valores como porcentagem (multiplicar por 100)
    percent_columns = ['x6', 'x7', 'x8', 'x9', 'x10', 'x11']
    for col in percent_columns:
        if col in df.columns:
            df[col] = df[col] * 100  # Multiplicando por 100 para mostrar em porcentagem
            df[col] = df[col].apply(lambda x: f"{x:.2f}%")  # Formatando como percentual

    logging.info("Dados formatados com sucesso.")
    return df

# Função para exibir os dados de forma mais legível
def display_formatted_data(df):
    """
    Exibe o DataFrame de forma mais legível.

    Args:
        df (pd.DataFrame): DataFrame a ser exibido.

    Returns:
        None
    """
    display(df.head())

# Função para salvar os dados
def save_data(df, output_path):
    """
    Salva o DataFrame em um arquivo CSV.

    Args:
        df (pd.DataFrame): DataFrame a ser salvo.
        output_path (str): Caminho do arquivo para salvar os dados.

    Returns:
        None
    """
    try:
        df.to_csv(output_path, index=False)
        logging.info(f"Dados salvos com sucesso no arquivo {output_path}.")
    except Exception as e:
        logging.error(f"Erro ao salvar os dados: {e}")
        raise

# Pipeline principal para carregar e formatar os dados
def main_pipeline(file_path, output_path):
    """
    Executa o pipeline completo de carregamento e formatação dos dados.

    Args:
        file_path (str): Caminho para o arquivo CSV de entrada.
        output_path (str): Caminho para salvar o arquivo CSV limpo.

    Returns:
        pd.DataFrame: DataFrame formatado.
    """
    # Carregar dados
    df = load_data(file_path)

    # Tratar valores ausentes
    df = handle_missing_values(df)

    # Remover duplicatas
    df = handle_duplicates(df)

    # Normalizar colunas numéricas
    df = normalize_columns(df)

    # Renomear colunas
    columns_dict = {
        'X1': 'ID', 'X2': 'Idade', 'X3': 'Sexo', 'X4': 'Estado',
        'X4.1': 'Renda', 'X6': 'Gasto1', 'X7': 'Gasto2', 'X8': 'Gasto3',
        'X9': 'Gasto4', 'X10': 'Gasto5', 'X11': 'Gasto6'
    }
    df = rename_columns(df, columns_dict)

    # Melhorar a formatação dos dados
    df = format_data(df)

    # Exibir dados formatados
    display_formatted_data(df)

    # Salvar os dados limpos
    save_data(df, output_path)

    return df

# Executar o pipeline
if __name__ == "__main__":
    file_path = "dados.csv"  # Substitua pelo caminho do seu arquivo CSV
    output_path = "dados_limpos.csv"  # Caminho para salvar o arquivo limpo
    main_pipeline(file_path, output_path)
